In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from utils import *
warnings.filterwarnings('ignore')
%matplotlib inline


In [8]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")
all_images = fd_collection.find()

In [9]:
selected_latent_space = valid_latent_spaces[
    str(input("Enter latent space - one of " + str(list(valid_latent_spaces.keys()))))
]

selected_feature_model = valid_feature_models[
    str(input("Enter feature model - one of " + str(list(valid_feature_models.keys()))))
]

k = int(input("Enter value of k (no. of latent semantics): "))
if k < 1:
    raise ValueError("k should be a positive integer")

k_2 = int(input("Enter value of k_2 (no. of similar images): "))
if k_2 < 1:
    raise ValueError("k_2 should be a positive integer")

if selected_latent_space != "cp":
    selected_dim_reduction_method = str(
        input(
            "Enter dimensionality reduction method - one of "
            + str(list(valid_dim_reduction_methods.keys()))
        )
    )

label = int(input("Enter label: "))
if label < 0 and label > 100:
    raise ValueError("label should be between 0 and 100")

label_rep = calculate_label_representatives(
    fd_collection, label, selected_feature_model
)


In [10]:
# Loading latent semantics
match selected_latent_space:
    # LS1
    case "":
        file_prefix = f"{selected_feature_model}-{selected_dim_reduction_method}-{k}"
        file_name = file_prefix + "-semantics.json"
        model_name = file_prefix + "-model.joblib"
        if os.path.exists(file_name):
            data = json.load(open(file_name))
            print(file_name + " loaded")
        else:
            raise Exception(file_name + " does not exist")
        # LDA model
        if selected_dim_reduction_method == "lda":
            if os.path.exists(model_name):
                data_model = load(model_name)
                print(model_name + " loaded")
            else:
                raise Exception(model_name + " does not exist")
    # LS2
    case "cp":
        file_name = f"{selected_feature_model}-cp-{k}-semantics.json"
        if os.path.exists(file_name):
            data = json.load(open(file_name))
            print(file_name + " loaded")
        else:
            raise Exception(file_name + " does not exist")
    # LS3, LS4
    case _:
        file_name = f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"
        if os.path.exists(file_name):
            data = json.load(open(file_name))
            print(file_name + " loaded")
        else:
            raise Exception(file_name + " does not exist")


cm_fd-svd-10-semantics.json loaded


In [11]:
def extract_similarities_ls1_ls4(latent_space, dim_reduction, data, label, label_rep):

    match dim_reduction:

        case "svd":
            U = np.array(data["image-semantic"])
            S = np.array(data["semantics-core"])
            if len(S.shape) == 1:
                S = np.diag(S)
            V = np.transpose(np.array(data["semantic-feature"]))

            if latent_space == "image_sim":
                label_vectors = []
                length = len(U)
                for i in range(length):
                    if all_images[i]["true_label"] == label:
                        label_vectors.append(U[i])
                label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]
                comparison_vector = np.matmul(label_rep, S)
            else:
                comparison_vector = np.matmul(np.matmul(label_rep, V), S)

            comparison_feature_space = np.matmul(U, S)

        case "nmf":
            H = np.array(data["semantic-feature"])
            comparison_feature_space = W = np.array(data["image-semantic"])
            if latent_space == "image_sim":
                label_vectors = []
                length = len(W)
                for i in range(length):
                    if all_images[i]["true_label"] == label:
                        label_vectors.append(W[i])
                label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]
                comparison_vector = label_rep
            else:
                min_value = np.min(label_rep)
                feature_vectors_shifted = label_rep - min_value
                comparison_vector = nmf(feature_vectors_shifted, H, update_H=False)

        case "kmeans":
            comparison_vector = []
            comparison_feature_space = np.array(data["image-semantic"])
            S = np.array(data["semantic-feature"])

            if latent_space == "image_sim":
                sim_matrix = np.array(data["sim-matrix"])
                label_vectors = []
                length = len(sim_matrix)
                for i in range(length):
                    if all_images[i]["true_label"] == label:
                        label_vectors.append(sim_matrix[i])
                label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]

            # get label_rep's kmeans semantic
            for centroid in S:
                comparison_vector.append(math.dist(label_rep, centroid))

        case "lda":

            comparison_feature_space = np.array(data["image-semantic"])
            if latent_space == "image_sim":
                label_vectors = []
                length = len(comparison_feature_space)
                for i in range(length):
                    if all_images[i]["true_label"] == label:
                        label_vectors.append(comparison_feature_space[i])
                label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]
                comparison_vector = label_rep
            else:
                min_value = np.min(label_rep)
                feature_vectors_shifted = label_rep - min_value
                comparison_vector = data_model.transform(
                    feature_vectors_shifted.flatten().reshape(1, -1)
                ).flatten()

    distances = []
    for i in range(NUM_IMAGES):
        distances.append(
            {
                "image_id": i,
                "label": all_images[i]["true_label"],
                "distance": math.dist(comparison_vector, comparison_feature_space[i]),
            }
        )

    distances = sorted(distances, key=lambda x: x["distance"], reverse=False)[:k_2]

    for x in distances:
        print(x)


In [12]:
def extract_similarities_ls2(data, label):

    LS_f = np.array(data["feature-semantic"])
    LS_i = np.array(data["image-semantic"])
    S = np.array(data["semantics-core"])

    if len(S.shape) == 1:
        S = np.diag(S)

    comparison_feature_space = np.matmul(label_rep, LS_f)
    comparison_vector = np.matmul(comparison_feature_space, S)

    comparison_image_space = np.matmul(LS_i, S)
    distances = []

    for i in range(NUM_IMAGES):
        distances.append(
            {
                "image": i,
                "distance": math.dist(comparison_vector, comparison_image_space[i]),
            }
        )

    distances = sorted(distances, key=lambda x: x["distance"], reverse=False)[:k_2]

    for x in distances:
        print(x)


In [13]:
def extract_similarities_ls3(dim_reduction, data, label):

    if dim_reduction == "svd":
        U = np.array(data["image-semantic"])
        S = np.array(data["semantics-core"])
        V = np.transpose(np.array(data["semantic-feature"]))

        comparison_feature_space = np.matmul(U, S)
    else:
        comparison_feature_space = np.array(data["image-semantic"])

    comparison_vector = comparison_feature_space[label]

    distances = []
    for i in range(NUM_LABELS):
        if i != label:
            distances.append(
                {
                    "label": i,
                    "distance": math.dist(
                        comparison_vector, comparison_feature_space[i]
                    ),
                }
            )
    
    most_similar_label = sorted(distances, key=lambda x: x["distance"], reverse=False)[0]
    print(f"Most similar label is {most_similar_label}")

    similar_images = []
    for i in range(len(dataset)):
        _, l = dataset[i]
        if l == label:
            similar_images.append(i)

    similar_images = random.sample(similar_images, k_2)
    images_distances = []
    for i in similar_images:
        images_distances.append(
            {"image_id": i, "distance": most_similar_label["distance"]}
        )

    for x in images_distances:
        print(x)
        


In [14]:
match selected_latent_space:

  case "" | "image_sim":
    
    extract_similarities_ls1_ls4(selected_latent_space, selected_dim_reduction_method, data, label, label_rep)

  case "label_sim":

    extract_similarities_ls3(selected_dim_reduction_method, data, label)

  case "cp":

    extract_similarities_ls2(data, label)
    

{'image_id': 499, 'label': 2, 'distance': 0.5891843615223927}
{'image_id': 449, 'label': 2, 'distance': 0.6183329800988425}
{'image_id': 462, 'label': 2, 'distance': 0.7954630378173778}
{'image_id': 512, 'label': 2, 'distance': 0.8431996693479317}
{'image_id': 506, 'label': 2, 'distance': 0.8541263603745314}
{'image_id': 438, 'label': 2, 'distance': 0.9166483319951415}
{'image_id': 491, 'label': 2, 'distance': 0.9340236427529084}
{'image_id': 527, 'label': 2, 'distance': 0.9349318595824383}
{'image_id': 441, 'label': 2, 'distance': 0.9351164972683086}
{'image_id': 490, 'label': 2, 'distance': 0.9440402757056761}
